In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.optimize import minimize
from scipy.stats import gmean

rets=pd.read_excel('50return3.xlsx')
rets=np.array(rets)
def gerber_cov_stat2(rets: np.array, threshold: float=0.5) -> tuple:
    """
    compute Gerber covariance Statistics 2
    :param rets: assets return matrix of dimension n x p
    :param threshold: threshold is between 0 and 1
    :return: Gerber covariance matrix of p x p
    """
    n, p = rets.shape
    sd_vec = rets.std(axis=0)
    U = np.copy(rets)
    D = np.copy(rets)

    # update U and D matrix
    for i in range(p):
        U[:, i] = U[:, i] >= sd_vec[i] * threshold
        D[:, i] = D[:, i] <= -sd_vec[i] * threshold

    # update concordant matrix
    N_CONC = U.transpose() @ U + D.transpose() @ D

    # update discordant matrix
    N_DISC = U.transpose() @ D + D.transpose() @ U
    H = N_CONC - N_DISC
    h = np.sqrt(H.diagonal())

    # reshape vector h and sd_vec into matrix
    h = h.reshape((p, 1))
    sd_vec = sd_vec.reshape((p, 1))

    cor_mat = H / (h @ h.transpose())
    cov_mat = cor_mat * (sd_vec @ sd_vec.transpose())
    return cov_mat, cor_mat
def minimize_volatility(weights, cov_matrix):
    """
    Returns the portfolio volatility given the weights and the covariance matrix
    """
    m_v=portfolio_volatility(weights, cov_matrix)
    return m_v
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.03})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
def portfolio_returns(returns, weights):
    weights = weights.transpose()
    p_r= np.dot(returns,weights)*12
    return p_r
def portfolio_volatility(weights, cov_matrix):
    """
    Returns the portfolio volatility given the weights and the covariance matrix
    """
    p_v=np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return p_v 
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=gerber_cov_stat2(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.03911862131357692
0.07834316081688454
0.09215359119408294
0.07835479581393
0.11580443237504084
0.11175929676070231
0.11219417658388044
0.17971100551789632
0.19853899921159956
0.23157679925122468
0.12812126087448844
0.11130016056231731
0.11713497226769687
0.11959103480770479
0.11161481901948843
0.10070898647684033
0.12101027587170018
0.10428232013264535
0.11341934396828057
0.11860190986562533
0.10757277458022733
0.11978852716643433
0.092695438567404
0.07996020766063638
0.09137932639990735
0.07612992297005629
0.06217911025394301
-0.0016653512093274803
0.039575306141573834
0.03895141565003806
0.07804549226228709
0.05414275417742329
0.06348918016559972
0.003520059600653949
0.10838736914595198
0.09945720857506257
0.13174721776455217
0.13022405739953233
0.10436472126402083
0.1118271450576438
0.09854172927547393
0.08907727183911748
0.10971794997930807
0.10299171855830022
0.13354724288714548
0.1367369857367899
0.121659379092558
0.10407943435671323
0.09823334484911751
0.10703209314128877
0.10

C:\Users\User\AppData\Local\Temp\ipykernel_5108\1212429032.py:38: RuntimeWarning: invalid value encountered in true_divide
  cor_mat = H / (h @ h.transpose())


0.1359664297320837
0.14774554672200116
0.11684832315029328
0.16295743759101347
0.1719655416190491
0.13885359367294306
0.15455749933120505
0.11437954448972325
0.11314530998874331
0.1266406541057833
0.09973548184298696
0.059127820995793515
0.027709808367776118
0.005495586674471166
0.0032565160526370307
0.014665533467403945
0.015701686827644147
-0.03724371839798696
-0.0132787865060579
0.0005609889593591629
0.01800774271979754
0.005470897883488069
0.007000997214717163
-0.021634371590791193
0.007974549147779671
0.010590022916738981
0.038709724856340555
0.030086228140694282
0.019461352689909533
0.012037108014207914
0.02547779945490541
0.0898859871574099
0.05525827045102071
0.1080897869556005
0.09983121053035285
0.12979861742300983
0.14769369282535716
0.17367203787833113
0.13553867547667278
0.15624003027176708
0.15187818047266277
0.16964988188023672
0.17181363761466736
0.1867595744232425
0.13621696691826182
0.22426878884144028
0.20389339857739086
0.21384845180023396
0.16816324716874947
0.1472

In [2]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.06})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=gerber_cov_stat2(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.048686313540953124
0.09692289331465441
0.10909519985546806
0.14684011318525744
0.19666449637346287
0.20745618447394523
0.26054234767355394
0.3758319462406382
0.4008989475423757
0.45947555453231237
0.3157545753734459
0.27834246458168715
0.2990820606624656
0.2716035117942794
0.26426145293490616
0.2971354273527967
0.31582002735913245
0.30903371271707897
0.3262973872243299
0.3378229927056189
0.28570762844870684
0.3255471782638895
0.2624129221567324
0.20940678447405905
0.2898356009134136
0.2800879377597349
0.273655369171795
0.16227283862591096
0.09334608278326849
0.14102697467881717
0.12182611160827359
0.07690262103212639
0.11302643932900368
0.06197947837410189
0.15371712630771675
0.14870090843685518
0.25184137464954454
0.22505053853927243
0.1868902509287959
0.22043669266369778
0.11090693762936206
0.14816786053769573
0.14192312068898846
0.11812447071778773
0.2044307514067189
0.10832008183857986
0.21665094715348884
0.10250409444484648
0.21089121959263885
0.19150816526731548
0.0891210053444

C:\Users\User\AppData\Local\Temp\ipykernel_5108\1212429032.py:38: RuntimeWarning: invalid value encountered in true_divide
  cor_mat = H / (h @ h.transpose())


0.14165008790232866
0.14236055825370347
0.1437687714523404
0.11860718629259746
0.12388547607629667
0.07666532885809252
0.050754385125216954
0.006754082618569618
-0.02467715333691466
-0.007313206856530039
-0.01601289412690583
-0.021131173914393755
-0.020332447141540484
-0.01908831967738873
0.024107630676916802
0.006903360024656678
0.002959850394688568
0.0004644022540456313
-0.013564994579011516
-0.010804519458051998
0.007483930268316508
0.025863634150180566
0.045124906326019114
0.029831137314345682
0.010275541245976938
0.028242886305322533
0.05533694978641126
0.03923304882058711
0.08573871454641019
0.1098245179837545
0.09864581851250946
0.14486143208171762
0.14253164158704887
0.18613487401999365
0.1661714555541267
0.1633494742770027
0.17791880946503794
0.19161685007573756
0.22003047357722555
0.14489626605943504
0.21994206326920912
0.18939140148444045
0.19989962898430802
0.15495499045612418
0.1407315182272863
0.12669868371687135
0.10789890848477467
0.1345234557743694
0.15256603278432923


In [3]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.09})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=gerber_cov_stat2(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.05449482870367587
0.1946375422807476
0.1495846223088393
0.15811591207857506
0.24537126555204378
0.3322553499609132
0.4087127317646523
0.5384753993708665
0.5581979553798669
0.6429516747381553
0.48859333094672175
0.4514249868061234
0.45774802976797
0.4478922094438725
0.42590316038965526
0.43634565106842144
0.4915986371571589
0.4876659716960233
0.5525475010349447
0.5099060595507446
0.43757518705926096
0.4241865852603814
0.4207043089015619
0.3854824860593385
0.4450583039421803
0.4212803151464315
0.3067799943493859
0.24157437835136011
0.2507156205307096
0.2997288567657421
0.19599955986729625
0.12022154767955796
0.13457474116049187
0.01995474283254071
0.22943429188614234
0.18641933664383434
0.2650454148439427
0.3739919302224506
0.3230908214677349
0.3798284421971647
0.20265496045407783
0.22299305501920494
0.23663353617137312
0.25548824114234564
0.3026061064026775
0.3399334227653837
-0.010003330317261772
0.33957549582420365
0.38809719068441956
0.048234138105335334
0.04555843055032692
0.12152

C:\Users\User\AppData\Local\Temp\ipykernel_5108\1212429032.py:38: RuntimeWarning: invalid value encountered in true_divide
  cor_mat = H / (h @ h.transpose())


0.18167195124941257
0.16454655069643082
0.14501381096308583
0.12575904720115702
0.16029170815744842
0.16978172497652594
0.17133675229603726
0.09012098194175183
0.12774557188133912
0.06645576698209565
0.06160680752535784
0.02884208092439025
0.019267015779256144
0.002681281973708773
0.031162414007248014
0.019513481454444814
-0.03369052192737004
0.012624127808658632
0.008462391408791526
-0.0139063941834689
0.042072289209111546
0.06695291668444828
0.0445794033844128
0.047553945303001895
0.056148066010458304
0.04241602530377034
0.025555069440480253
0.10304447657942344
0.05397868779538713
-0.007517868708572388
-0.0009735051506029703
0.06465097121680785
0.03133740807138322
0.10625273065035595
0.15116236398272717
0.20030955547685605
0.19278053632655154
0.21834621034913032
0.25407511822707246
0.30468147938708684
0.39067201304202454
0.32049269910391126
0.2953430041085057
0.2597553999791908
0.1827045100771502
0.06168646844963541
0.0687081572539905
0.018164306224635884
-0.07512048564901139
-0.0304

In [4]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.12})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=gerber_cov_stat2(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.07916278531361517
0.15180925918920313
0.2023729360348024
0.24046439882613657
0.2890544108696279
0.3693693585124358
0.5233590842064106
0.6763470069938808
0.7139670879580982
0.8009980610362083
0.6340595664785997
0.5808980536348798
0.6066510400237795
0.6182768347902431
0.5466851653185218
0.6527474762671844
0.6647450660577183
0.6389982326801263
0.7359340944601243
0.6793471886183662
0.5979560415397003
0.5688831897651758
0.5239007033782659
0.42839008018735397
0.4860425604901953
0.2474379074597103
0.19879869951141
0.504785063041933
0.18166706528146376
0.10899402586893572
0.46734183345992014
0.3682874120261861
0.05929505605672888
-0.03584144045014894
0.3499218800022091
0.1783565522326331
0.26492111297238596
0.522486074533934
0.483920244696139
0.46731665227669716
0.3960496341622557
0.2533279113577205
0.33489293513766805
0.28923968343995166
0.3167597457959468
0.41049039321955394
0.45237928074034656
0.42568760257294025
0.48903985350705814
0.022668257101178233
0.022305871201923427
0.154463024933

C:\Users\User\AppData\Local\Temp\ipykernel_5108\1212429032.py:38: RuntimeWarning: invalid value encountered in true_divide
  cor_mat = H / (h @ h.transpose())


0.001226257466168768
0.03964456952823826
0.07995965522371111
0.04871944640936336
0.06510318471624199
0.09522536942832353
0.10109893313246665
0.13713177122246747
0.1882369116278202
0.1662646707951736
0.2037522919181622
0.1843461317152515
0.19725244182516896
0.20918619777196515
0.23188688257150883
0.15692252211078306
0.21517079036357198
0.18189891759074178
0.18509346622486247
0.1804994573166228
0.1704317708811992
0.13617270690436567
0.09133442589574699
0.1171314092673123
0.18552210210407286
0.16589700001356533
0.13167171303756917
0.1291887422537555
0.07386350194004425
0.03432813041895436
0.022221454647948342
0.02686262803811595
-0.02430042186059221
-0.010313659491340332
0.026625058817186274
-0.006959986317691961
-0.04120883879114728
0.027493521444052458
-0.03596200109147971
0.057756592549496666
0.045930077555619966
0.030427153437148708
0.026686731387293276
0.0764969025498344
0.05066402288417473
0.08969857581145273
0.09964660404797804
0.07926037378424981
0.0011737356485141063
0.0559189457

In [5]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.15})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=gerber_cov_stat2(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.0895709438621445
0.25792416706439236
0.21324819286097754
0.31245268242980023
0.40898634939314676
0.46132266229185226
0.6830569041772165
0.8614013085820384
0.8122384579847246
0.9337569950631179
0.7568894186643657
0.7229080943062534
0.7965018565177908
0.7316597769385971
0.6554937749501445
0.7791494087599601
0.7838829865976904
0.7374511298351514
0.9184800038407827
0.8232501398537623
0.7612825583993366
0.6088028399339576
0.5124035513804811
0.3980875612076398
0.5078899138806239
0.20822175214864394
0.1757187924612995
0.027204008674417954
-0.1240475523903749
-0.1147344491515034
-0.19997848814017172
-0.1725263875314741
0.015436917218736882
-0.06544904392473899
0.26227635996545734
0.17789917205729694
0.27896084705423213
0.5638396521421821
0.2856571722002899
0.5082713283014465
0.43081817730155336
0.284177830647563
0.3666490296001584
0.3277872552460441
0.38705725508425426
0.487466901497221
-0.00872161949662871
0.5000370689396311
0.576710547639874
0.6366099572212235
0.5371607082995231
0.14194466

C:\Users\User\AppData\Local\Temp\ipykernel_5108\1212429032.py:38: RuntimeWarning: invalid value encountered in true_divide
  cor_mat = H / (h @ h.transpose())


-0.009592655436582564
0.00818898931102698
-0.012592613264242295
0.008187524912174355
-0.023704946335065667
0.0037217673848878277
0.012955468912839715
0.02467338029197063
-0.0036642655872242734
0.01634345502899527
0.011366273331057405
0.044823672398821984
0.06526370923284991
0.05446505682699876
0.09471925976129146
0.10046871968433499
0.10803953315467818
0.14729831309418503
0.15056020619672802
0.18314831779921867
0.1661466192754334
0.16255485459318145
0.17477165228495334
0.1834365376057563
0.17815840538862832
0.1345535841007549
0.2049101961915991
0.1870178146889056
0.20804570325898084
0.18095999325857573
0.1617065049050362
0.14738282617541534
0.08034226741899195
0.14510317398220507
0.17097086746761528
0.13412778806049933
0.10872875252075277
0.13450460318627352
0.073907974197607
0.03503308117985407
0.05728098787635843
0.03282404942806714
0.01384126299260597
0.03518789529906946
0.02378826076790128
-0.00576057442464988
-0.05181928694343484
0.014847426286462101
-0.01704557005130664
-0.002568